In [1]:
import collections

import wandb
import time
import numpy as np
import os
import cv2

from PIL import Image

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader


from models.retina.retinanet import model, coco_eval
from models.retina.analyze_regression_moe import analyze_regression, load_classes
from models.retina.retinanet.dataloader import CocoDataset, CocoSubDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from models.moe_layer.soft_gating_networks import FCGate

In [2]:
# Create the model
retinanet = model.resnet50(num_classes=80,)
retinanet.load_state_dict(torch.load('/home/lb4653/mixture-of-experts-thesis/baseline_state_dict.pt'))

# Freeze all layers
for param in retinanet.parameters():
    param.requires_grad = False

# Add MoE Predictors
gate1 = FCGate(in_channels=256, 
                    num_experts=4,
                    top_k=2,
                    use_noise=True,
                    name='FCGate',
                    loss_fkt='kl_divergence',
                    w_aux_loss=0.25
                    )

gate2 = FCGate(in_channels=256, 
                num_experts=4,
                top_k=2,
                use_noise=True,
                name='FCGate',
                loss_fkt='kl_divergence',
                w_aux_loss=0.25
                )

retinanet.classificationModel = model.ClassificationModelMoE(num_features_in=256, num_experts=4, top_k=2, gating_network=gate1, num_classes=80)
retinanet.regressionModel = model.RegressionModelMoE(num_features_in=256, num_experts=4, top_k=2, gating_network=gate2)

file_model = wandb.restore('retinanet_final.tar', run_path='lukas-struppek/RetinaNet/16q9vpvj')
retinanet.load_state_dict(torch.load('retinanet_final.tar')['model_state_dict'])

<All keys matched successfully>

In [3]:
analyze_regression('/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/tenet_autos', retinanet, '/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results', class_list='/home/lb4653/mixture-of-experts-thesis/data/coco/coco.names', active_experts=True, inactivate_experts=False, expert_colors=[(240, 32, 160), (0, 255, 136), (0, 45, 255), (0, 45, 255)], scale_factor=1)

anet/image_results/out301_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out157_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out219_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out340_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out351_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out176_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out174_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out60_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out195_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out308_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out235_pred.png

/home/lb4653/mixture-of-experts-thesis/analysis/retinanet/image_results/out